# Chain of Responsibility Pattern

- __Type:__ Behavioral
- __Popularity: ★★★☆☆__
- __Complexity: ★★☆☆☆__

The Chain of Responsibility pattern passes a request along a chain of handlers. Upon receiving a request, each handler decides either to process the request or to pass it to the next handler in the chain.

### Intent:

Chain of Responsibility is a behavioral design pattern that lets you pass requests along a chain of handlers. Upon receiving a request, each handler decides either to process the request or to pass it to the next handler in the chain.

### Problem:

You need to process a request through multiple handlers, but you don't want to hardcode the sequence or make the client aware of the chain's structure. Specific challenges include:

- Multiple objects may handle a request, but the handler isn't known in advance
- You want to issue a request to one of several objects without specifying the receiver explicitly
- The set of objects that can handle a request should be specified dynamically
- Hard-coding handler relationships leads to tight coupling between sender and receiver

### Solution:

The Chain of Responsibility pattern suggests organizing handlers into a chain where each handler has a reference to the next handler. The request travels down the chain until a handler processes it or it reaches the end of the chain.

Key components:
- **Handler**: Defines an interface for handling requests and maintaining a successor link
- **ConcreteHandler**: Handles requests it is responsible for and passes others to its successor
- **Client**: Initiates the request to the first handler in the chain

This approach decouples senders and receivers, allowing multiple objects to handle the request without the sender needing to know which object will ultimately process it.

### Diagram:

```mermaid
classDiagram
    class Handler {
        +successor: Handler
        +handle_request(request)
    }
    class ConcreteHandlerA {
        +handle_request(request)
    }
    class ConcreteHandlerB {
        +handle_request(request)
    }
    class Client {
        +send_request(request)
    }
    
    Handler <|-- ConcreteHandlerA
    Handler <|-- ConcreteHandlerB
    Handler o-- Handler: successor
    Client ..> Handler
```

### Example code:

In [5]:
class Handler:
    def __init__(self, successor=None):
        self.successor = successor  # Next handler in the chain

    def handle_request(self, request):
        if self.successor:
            return self.successor.handle_request(request)
        return None


class ConcreteHandlerA(Handler):
    def handle_request(self, request):
        if request == "A":
            return "Handled by Handler A"
        return super().handle_request(request)


class ConcreteHandlerB(Handler):
    def handle_request(self, request):
        if request == "B":
            return "Handled by Handler B"
        return super().handle_request(request)


# Setting up the chain
handler_chain = ConcreteHandlerA(ConcreteHandlerB())

# Testing the chain
print(handler_chain.handle_request("A"))  # Output: Handled by Handler A
print(handler_chain.handle_request("B"))  # Output: Handled by Handler B
print(handler_chain.handle_request("C"))  # Output: None

Handled by Handler A
Handled by Handler B
None


In this simple example:

1. We define a base `Handler` class with a successor reference and a `handle_request` method
2. Concrete handlers check if they can process the request; if not, they pass it to the next handler
3. The client creates a chain by linking handlers and sends requests to the first handler

## Real-World Example: Support Ticket System

Let's implement a support ticket system where tickets are handled by different support levels based on priority:

In [6]:
class SupportTicket:
    def __init__(self, id, customer, issue, priority):
        self.id = id
        self.customer = customer
        self.issue = issue
        self.priority = priority  # 1 (low) to 3 (high)

    def __str__(self):
        return f"Ticket #{self.id}: {self.issue} ({self.customer}) - Priority: {self.priority}"

In [7]:
class SupportHandler:
    def __init__(self, name, successor=None):
        self.name = name
        self.successor = successor

    def handle_ticket(self, ticket):
        if self.successor:
            return self.successor.handle_ticket(ticket)
        return f"Ticket #{ticket.id} could not be handled by any support level"


class FirstLevelSupport(SupportHandler):
    def handle_ticket(self, ticket):
        if ticket.priority == 1:
            return f"{self.name} handled ticket #{ticket.id}: {ticket.issue}"
        print(f"{self.name}: Escalating ticket #{ticket.id} (priority {ticket.priority})")
        return super().handle_ticket(ticket)


class SecondLevelSupport(SupportHandler):
    def handle_ticket(self, ticket):
        if ticket.priority <= 2:
            return f"{self.name} handled ticket #{ticket.id}: {ticket.issue}"
        print(f"{self.name}: Escalating ticket #{ticket.id} (priority {ticket.priority})")
        return super().handle_ticket(ticket)


class ProductSpecialist(SupportHandler):
    def handle_ticket(self, ticket):
        return f"{self.name} handled ticket #{ticket.id}: {ticket.issue}"

In [8]:
# Create the support chain
support_chain = FirstLevelSupport(
    "Help Desk", SecondLevelSupport("Technical Support", ProductSpecialist("Product Specialist"))
)

# Create some support tickets
tickets = [
    SupportTicket(1, "John Doe", "Password reset", 1),
    SupportTicket(2, "Jane Smith", "Software installation issue", 2),
    SupportTicket(3, "Bob Johnson", "Server down", 3),
]

# Process tickets through the support chain
for ticket in tickets:
    print(f"\nProcessing: {ticket}")
    result = support_chain.handle_ticket(ticket)
    print(f"Result: {result}")


Processing: Ticket #1: Password reset (John Doe) - Priority: 1
Result: Help Desk handled ticket #1: Password reset

Processing: Ticket #2: Software installation issue (Jane Smith) - Priority: 2
Help Desk: Escalating ticket #2 (priority 2)
Result: Technical Support handled ticket #2: Software installation issue

Processing: Ticket #3: Server down (Bob Johnson) - Priority: 3
Help Desk: Escalating ticket #3 (priority 3)
Technical Support: Escalating ticket #3 (priority 3)
Result: Product Specialist handled ticket #3: Server down


### Real-world analogies:

1. **Customer Service Escalation:**
   When you call customer service, your issue might start with a frontline representative. If they cannot solve it, they escalate to a supervisor, then to a manager, and so on until your issue is resolved.
   - The call center representatives = handlers in the chain
   - Your issue = the request
   - The escalation process = passing the request along the chain

2. **Office Approval Process:**
   When requesting approval for expenses, your request might start with a team leader, move to a department manager, and potentially end with a CFO depending on the amount.
   - Each approving authority = a handler
   - The expense request = the request being passed
   - Approval thresholds = conditions for handling

## When to Use

* When multiple objects may handle a request, but the handler isn't known in advance
* When you want to issue a request to one of several objects without specifying the receiver explicitly
* When the set of objects that can handle a request should be specified dynamically

### Python-specific implementation notes:

- Python's duck typing allows for more flexible handler interfaces compared to strictly typed languages
- Python's method resolution order (MRO) and `super()` function make it easy to implement the chain logic in subclasses
- Consider using `@abc.abstractmethod` to define handler interfaces formally
- Python's context managers (`with` statement) can sometimes be an alternative way to handle sequential processing
- For more complex scenarios, consider using middleware patterns as seen in Django or other Python web frameworks

## Benefits of the Chain of Responsibility Pattern

* **Decoupling**: Separates the sender of a request from its receivers
* **Flexibility**: Makes it easy to add new handlers or change the order of handlers
* **Single Responsibility**: Each handler focuses only on its specific responsibility
* **Dynamic Construction**: Chains can be built dynamically at runtime based on specific needs
* **Reduced Conditional Complexity**: Eliminates complex conditional statements by distributing decisions across handlers

### Related patterns:

- **Composite Pattern**: Often used with Chain of Responsibility when requests need to be passed to a tree structure of handlers
- **Command Pattern**: Commands can be chained, and the Chain of Responsibility can decide which command to execute
- **Mediator Pattern**: Provides an alternative approach to reducing direct connections between objects
- **Observer Pattern**: Sometimes used alongside Chain of Responsibility when multiple handlers might need to react to an event